In [1]:
from raise_utils.interpret import DODGEInterpreter
from typing import Union, Callable
import numpy as np

In [2]:
# Order matters!
files = ["ivy", "lucene", "poi", "synapse", "velocity", "camel", "jedit", "log4j", "xalan", "xerces"]

In [3]:
class DODGEInterpreter:
    """Interprets the results of DODGE-generated files"""

    def __init__(self, files=None, n_datasets=1, max_by: Union[None, int, Callable[..., str]] = None,
                 exclude_cols=None, metrics=None) -> None:
        """
        Initializes the interpreter.
        :param files - A list of files to be interpreted.
        :param max_by - Either a None, int, or Callable. If None, defaults to
                        maximizing the first entry, the metric maximized by DODGE.
                        If int, maximizes by the index specified.
                        If callable, maximizes by the function passed.
        :param exclude_cols - List of column indices to exclude
        :param metrics - List of metrics passed to DODGE. If excluding columns,
                        do NOT include these in this list.
        :return DODGEInterpreter object
        """
        if files is None:
            files = []
        if exclude_cols is None:
            exclude_cols = []
        if metrics is None:
            metrics = []
        self.files = files
        self.n_datasets = n_datasets
        if max_by is None:
            self.max_by = 0
        else:
            self.max_by = max_by
        self.exclude_cols = exclude_cols
        self.metrics = metrics

    def interpret(self) -> dict:
        DODGE_ITER = 30
        medians = {}

        for file in self.files:
            with open(file, 'r') as f:
                lines = f.readlines()
            
            popt_lines = lines.copy()
            popt_lines = [float(x.split(':')[1]) for x in popt_lines if x.startswith('popt20')]
            popt_lines = np.array(popt_lines).reshape((self.n_datasets, 20, DODGE_ITER))

            settings = [line.split(':')[1]
                        for line in lines if line.startswith('setting')]

            lines = [eval(line.split(':')[1])
                     for line in lines if line.startswith('iter')]

            n_runs = int(len(lines) // (DODGE_ITER * self.n_datasets))
            n_metrics = len(lines[0]) - len(self.exclude_cols)

            if len(self.metrics) == 0:
                self.metrics = list(range(n_metrics))
            elif len(self.metrics) != n_metrics:
                raise ValueError("Passed list of metrics has size", len(self.metrics),
                                 "but file metrics (excluding exclude_cols) has size",
                                 n_metrics)

            lines = np.array(lines)
            lines = np.delete(lines, self.exclude_cols, -1)

            settings = np.array(settings)

            assert lines.shape == (n_runs * self.n_datasets * DODGE_ITER, n_metrics)
            
            run_splits = lines.reshape(
                (self.n_datasets, n_runs, DODGE_ITER, n_metrics))
            settings = settings.reshape((self.n_datasets, n_runs, DODGE_ITER))

            if isinstance(self.max_by, int):
                mapped_vals = np.apply_along_axis(
                    lambda x: x[self.max_by], axis=-1, arr=run_splits).reshape(self.n_datasets, n_runs, DODGE_ITER)
            elif callable(self.max_by):
                mapped_vals = np.apply_along_axis(
                    self.max_by, axis=-1, arr=run_splits).reshape(self.n_datasets, n_runs, DODGE_ITER)

            assert mapped_vals.shape == (self.n_datasets, n_runs, DODGE_ITER)

            max_idx = np.argmax(mapped_vals, axis=-1)
            
            medians[file.split('/')[-1]] = {metric: max_idx.choose(np.rollaxis(np.apply_along_axis(lambda p: p[i], -1, run_splits), -1, 0))
                                            for i, metric in enumerate(self.metrics)}
            medians[file.split('/')[-1]]['setting'] = max_idx.choose(np.rollaxis(settings, -1, 0))
            medians[file.split('/')[-1]]['popt20'] = max_idx.choose(np.rollaxis(popt_lines, -1, 0))

        return medians

In [11]:
interp = DODGEInterpreter(files=['dodge.txt'], metrics=["d2h", "auc", "pd", "pf", "prec", "f1"], n_datasets=10)
res = interp.interpret()

In [12]:
res.keys()

dict_keys(['dodge.txt'])

In [13]:
res["dodge.txt"].keys()

dict_keys(['d2h', 'auc', 'pd', 'pf', 'prec', 'f1', 'setting', 'popt20'])

In [150]:
res["dodge.txt"]["d2h"].shape

(10,)

In [154]:
res["dodge.txt"]["auc"]

array([0.72980769, 0.5981626 , 0.71326894, 0.57623119, 0.5914629 ,
       0.56491826, 0.69060669, 0.63144841, 0.66703786, 0.54252353])

In [153]:
res["dodge.txt"]["setting"]

array([[' maxabsV|rf\n', ' standardizeJ|rf\n', ' normalizee|rf\n',
        ' minmaxr|rf\n', ' minmaxt|rf\n', ' maxabsS|rf\n',
        ' maxabsW|rf\n', ' standardizef|rf\n', ' minmaxB|rf\n',
        ' normalizex|rf\n', ' minmaxD|rf\n', ' minmaxU|rf\n',
        ' minmaxB|rf\n', ' normalizep|rf\n', ' minmaxH|rf\n',
        ' maxabsC|rf\n', ' minmaxK|rf\n', ' standardizen|rf\n',
        ' normalizez|rf\n', ' minmaxF|rf\n'],
       [' standardizeO|rf\n', ' minmaxP|rf\n', ' normalizee|rf\n',
        ' minmaxD|rf\n', ' standardizef|rf\n', ' normalizeJ|rf\n',
        ' maxabst|rf\n', ' standardizeP|rf\n', ' maxabsR|rf\n',
        ' minmaxb|rf\n', ' standardized|rf\n', ' normalizeq|rf\n',
        ' minmaxJ|rf\n', ' minmaxS|rf\n', ' normalizeC|rf\n',
        ' normalizey|rf\n', ' minmaxJ|rf\n', ' minmaxf|rf\n',
        ' maxabsx|rf\n', ' normalizeZ|rf\n'],
       [' maxabsP|rf\n', ' normalizex|rf\n', ' normalizeQ|rf\n',
        ' minmaxq|rf\n', ' normalizej|rf\n', ' minmaxN|rf\n',
        ' stan

In [157]:
for i, name in enumerate(files):
    print(name + ":")
    print("=" * len(name + ":"))
    
    for key in res["dodge.txt"].keys():
        if key == "setting":
            continue
        
        print(key, "-", res["dodge.txt"][key][i])
    
    print()

ivy:
====
d2h - 0.42063608708350025
auc - 0.7298076923076923
pd - 0.825
pf - 0.36538461538461536
prec - 0.22448979591836735
f1 - 0.35294117647058826

lucene:
d2h - 0.3025384247038405
auc - 0.59816259753335
pd - 0.6379310344827587
pf - 0.43795620437956206
prec - 0.6818103818103818
f1 - 0.6590631626293131

poi:
====
d2h - 0.3888260540678474
auc - 0.7132689374682257
pd - 0.5765124555160143
pf - 0.14596273291925466
prec - 0.872983870967742
f1 - 0.6932154212668131

synapse:
d2h - 0.1923082282900893
auc - 0.5762311901504789
pd - 0.28488372093023256
pf - 0.13529411764705881
prec - 0.529457065845206
f1 - 0.3670971364402021

velocity:
d2h - 0.26291807925268096
auc - 0.5914628969264731
pd - 0.75
pf - 0.5695364238410596
prec - 0.40687883545026404
f1 - 0.5251162499739349

camel:
d2h - 0.17432390906452683
auc - 0.5649182617267724
pd - 0.2579787234042553
pf - 0.12612612612612611
prec - 0.3277921961696796
f1 - 0.2882927728613569

jedit:
d2h - 0.36535354170430934
auc - 0.6906066906066906
pd - 0.636363

In [158]:
interp = DODGEInterpreter(files=['ghost.txt'], metrics=["d2h", "auc", "pd", "pf", "prec", "f1"], n_datasets=10)
res = interp.interpret()

In [161]:
for i, name in enumerate(files):
    print(name + ":")
    print("=" * len(name + ":"))
    
    for key in res["ghost.txt"].keys():
        if key == "setting":
            continue
        
        print(key, "-", res["ghost.txt"][key][i])
    
    print()

ivy:
====
d2h - 0.4125309223626752
auc - 0.7301282051282051
pd - 0.825
pf - 0.35416666666666663
prec - 0.22327844311377246
f1 - 0.35616331619136665

lucene:
d2h - 0.2994001961807583
auc - 0.5985671137319766
pd - 0.6551724137931034
pf - 0.4635036496350365
prec - 0.6779352226720647
f1 - 0.6658228799859648

poi:
====
d2h - 0.332934330491189
auc - 0.6299374461218805
pd - 0.6192170818505338
pf - 0.34782608695652173
prec - 0.754860101744186
f1 - 0.6821657677688806

synapse:
d2h - 0.3698852826288851
auc - 0.6850205198358413
pd - 0.5755813953488372
pf - 0.20294117647058824
prec - 0.5913194444444445
f1 - 0.5806818181818182

velocity:
d2h - 0.2521430031533113
auc - 0.5605790456783835
pd - 0.6538461538461539
pf - 0.543046357615894
prec - 0.3854030501089325
f1 - 0.4879125499264242

camel:
d2h - 0.2657219639667243
auc - 0.5733402475423752
pd - 0.5132978723404256
pf - 0.3384813384813385
prec - 0.25636831913973646
f1 - 0.3319334304364245

jedit:
d2h - 0.32906236665304056
auc - 0.6242676242676243
pd -

In [13]:
def print_latex(dl_results, dodge_results, ghost_results):
    def get_dodge_cell(m, i):
        return f"{round(dodge_results['dodge.txt'][m][i], 2)}"
    
    def get_ghost_cell(m, i):
        return f"{round(ghost_results['ghost.txt'][m][i], 2)}"
    
    def get_dl_cell(m, i):
        return f"{round(dl_results[m][i], 2)}"
    
    print(r"&       & AUC  & popt20 & recall & pf  \\ \midrule")
    for i, name in enumerate(files):
        print(f"\multirow{{3}}{{*}}{'{' + name + '}'}\t& DL & {get_dl_cell('auc', i)} & & {get_dl_cell('pd', i)} & {get_dl_cell('pf', i)} \\\\")
        print(f"\t& DODGE & {get_dodge_cell('auc', i)} & & {get_dodge_cell('pd', i)} & {get_dodge_cell('pf', i)} \\\\")
        print(f"\t& GHOST & {get_ghost_cell('auc', i)} & & {get_ghost_cell('pd', i)} & {get_ghost_cell('pf', i)} \\\\", end=" ")
        
        if i == len(files) - 1:
            print()
        else:
            print(r"\midrule")

In [15]:
def parse_dl_results(metrics, n_datasets=10, n_runs=20):
    with open('dl.txt', 'r') as f:
        lines = f.readlines()
    
    #lines_metr = [eval(x.split(':')[1]) for x in lines if x.startswith('metr')]
    #print(lines_metr)
    #lines_metr = np.array(lines_metr).reshape(n_datasets, n_runs, len(metrics))
    
    #res = {metrics[i]: np.apply_along_axis(lambda p: p[i], -1, lines_metr) for i in range(len(metrics))}
    res = {}
    lines_popt = [float(x.split(':')[1]) for x in lines if x.startswith('popt20')]
    lines_popt = np.array(lines_popt).reshape(n_datasets, n_runs, 30)
    res['popt20'] = np.median(lines_popt, axis=-1)
    return res

In [16]:
dl_res = parse_dl_results(metrics=["d2h", "auc", "pd", "pf", "prec", "f1"])

In [23]:
dl_res['d2h'].shape

(10, 20)

In [16]:
interp = DODGEInterpreter(files=['dodge.txt'], metrics=["d2h", "auc", "pd", "pf", "prec", "f1"], n_datasets=10)
dodge_res = interp.interpret()

In [8]:
interp = DODGEInterpreter(files=['ghost.txt'], metrics=["d2h", "auc", "pd", "pf", "prec", "f1"], n_datasets=10)
ghost_res = interp.interpret()

In [24]:
dodge_res['dodge.txt']['d2h'].shape

(10, 20)

In [14]:
print_latex(dl_res, dodge_res, ghost_res)

NameError: name 'print_latex' is not defined

## Comparing to old results

In [4]:
from raise_utils.interpret.sk import Rx

In [5]:
def get_old_result(file, metric, algorithm):
    if metric == "pd":
        metric = "recall"
    
    filename = f'./unencoded-results/{file}_{metric}'
    
    if file == 'xalan' and metric == 'popt20':
        filename += '.txt'
    else:
        filename += '_new.txt'
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    line = [eval(f'[{x.split(algorithm + ",")[1]}]') for x in lines if x.startswith(algorithm)][0]
    return line

In [6]:
def get_new_result(i, metric, algorithm):
    if algorithm == 'DL':
        return dl_res[metric][i]
    if algorithm == 'DODGE':
        return dodge_res['dodge.txt'][metric][i]
    return ghost_res['ghost.txt'][metric][i]

In [14]:
dl_res

In [46]:
possible_results = ["WIN", "TIE", "LOSS"]

for i, file in enumerate(files):
    print(file)
    print('=' * len(file))
    
    for metric in ["auc", "pd", "pf", "popt20"]:
        print('\t' + metric, end='')
    
    print()
    for alg in ['DL', 'DODGE', 'GHOST']:
        print(alg, end='\t')
        
        for metric in ["auc", "pd", "pf", "popt20"]:
            old = get_old_result(file, metric, alg)
            new = get_new_result(i, metric, alg)
            
            data = {'old': old, 'new': new}
            results = Rx.sk(Rx.data(**data))
            
            result = ""
            # if results[0].rx == 'old':
            if results[1].rank > results[0].rank:
                if metric == 'pf':
                    # Old pf < new pf: loss
                    result = "LOSS"
                else:
                    result = "WIN"
            elif results[1].rank == results[0].rank:
                result = "TIE"
            else:
                if metric == 'pf':
                    result = "WIN"
                else:
                    result = "LOSS"
            
            if results[0].rx == 'new':
                # Flip the result
                idx = possible_results.index(result)
                result = possible_results[2 - idx]
            
            print(result, end='\t')
        print()
    
    print()

ivy
===
	auc	pd	pf
DL	LOSS	LOSS	WIN	
DODGE	WIN	LOSS	TIE	
GHOST	WIN	LOSS	WIN	

lucene
	auc	pd	pf
DL	WIN	LOSS	WIN	
DODGE	TIE	LOSS	LOSS	
GHOST	WIN	LOSS	LOSS	

poi
===
	auc	pd	pf
DL	LOSS	LOSS	WIN	
DODGE	LOSS	LOSS	WIN	
GHOST	LOSS	LOSS	TIE	

synapse
	auc	pd	pf
DL	LOSS	LOSS	WIN	
DODGE	LOSS	LOSS	WIN	
GHOST	WIN	LOSS	WIN	

velocity
	auc	pd	pf
DL	LOSS	WIN	LOSS	
DODGE	LOSS	TIE	LOSS	
GHOST	LOSS	LOSS	LOSS	

camel
=====
	auc	pd	pf
DL	TIE	TIE	WIN	
DODGE	LOSS	LOSS	WIN	
GHOST	LOSS	LOSS	WIN	

jedit
=====
	auc	pd	pf
DL	WIN	TIE	TIE	
DODGE	WIN	WIN	WIN	
GHOST	LOSS	TIE	TIE	

log4j
=====
	auc	pd	pf
DL	LOSS	LOSS	TIE	
DODGE	WIN	LOSS	WIN	
GHOST	LOSS	LOSS	LOSS	

xalan
=====
	auc	pd	pf
DL	TIE	TIE	WIN	
DODGE	LOSS	LOSS	WIN	
GHOST	LOSS	LOSS	WIN	

xerces
	auc	pd	pf
DL	TIE	TIE	TIE	
DODGE	LOSS	LOSS	WIN	
GHOST	LOSS	LOSS	WIN	



In [27]:
??Rx

## Interpreting results with popt20

In [9]:
possible_results = ["WIN", "TIE", "LOSS"]

for i, file in enumerate(files):
    print(file)
    print('=' * len(file))
    
    for metric in ["popt20"]:
        print('\t' + metric, end='')
    
    print()
    for alg in ['GHOST']:
        print(alg, end='\t')
        
        for metric in ["popt20"]:
            old = get_old_result(file, metric, alg)
            new = get_new_result(i, metric, alg)
            
            data = {'old': old, 'new': new}
            results = Rx.sk(Rx.data(**data))
            
            result = ""
            # if results[0].rx == 'old':
            if results[1].rank > results[0].rank:
                if metric == 'pf':
                    # Old pf < new pf: loss
                    result = "LOSS"
                else:
                    result = "WIN"
            elif results[1].rank == results[0].rank:
                result = "TIE"
            else:
                if metric == 'pf':
                    result = "WIN"
                else:
                    result = "LOSS"
            
            if results[0].rx == 'new':
                # Flip the result
                idx = possible_results.index(result)
                result = possible_results[2 - idx]

            print(result, f'({round(np.median(new), 2)})', end='\t')
        print()
    
    print()

ivy
===
	popt20
GHOST	WIN (0.6)	

lucene
	popt20
GHOST	LOSS (0.68)	

poi
===
	popt20
GHOST	TIE (0.72)	

synapse
	popt20
GHOST	WIN (0.63)	

velocity
	popt20
GHOST	LOSS (0.58)	

camel
=====
	popt20
GHOST	LOSS (0.43)	

jedit
=====
	popt20
GHOST	WIN (0.54)	

log4j
=====
	popt20
GHOST	LOSS (0.66)	

xalan
=====
	popt20
GHOST	LOSS (0.61)	

xerces
	popt20
GHOST	LOSS (0.61)	

